In [27]:
from transformers import AutoImageProcessor, ViTForImageClassification, ViTConfig, ViTModel

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = ViTForImageClassification.from_pretrained(checkpoint)
model.config.num_labels

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2

In [28]:
model.config.num_labels = 1
model.config.num_labels

1

In [30]:
image_processor = AutoImageProcessor(model.config)
model = ViTForImageClassification(model.config)

TypeError: __init__() takes 1 positional argument but 2 were given

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

size = (image_processor.size["height"], image_processor.size["width"])

print(size)

data_augmentation = keras.Sequential(
    [
        layers.Resizing(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
    ],
    name="data_augmentation",
)

(224, 224)


In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
b = np.load("../data/6-parameter-maps/X_maps_Cosmogrid_100k.npy", allow_pickle=True)
a = np.load("../data/6-parameter-maps/y_maps_Cosmogrid_100k.npy", allow_pickle=True)

num_samples = len(a)
train_split, val_split, test_split = int(0.80*num_samples), \
            int(0.10*num_samples), int(0.10*num_samples) + 1

train_x, val_x, test_x = np.split(b[:, :, :, None], [train_split, train_split+val_split])
train_y, val_y, test_y = np.split(a, [train_split, train_split+val_split])

output_num = 5

train_y, val_y, test_y = train_y[:,:output_num], val_y[:,:output_num], test_y[:,:output_num]

scaler = MinMaxScaler()
scaler.fit(train_y)
train_y, val_y, test_y = scaler.transform(train_y), scaler.transform(val_y), scaler.transform(test_y)

print(f"x_train shape: {train_x.shape} - y_train shape: {train_y.shape}")
print(f"x_test shape: {test_x.shape} - y_test shape: {test_y.shape}")


x_train shape: (80000, 66, 66, 1) - y_train shape: (80000, 5)
x_test shape: (10000, 66, 66, 1) - y_test shape: (10000, 5)


In [13]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 5
num_train_steps = len(train_x) * num_epochs
learning_rate = 3e-5
weight_decay_rate = 0.01

optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0,
)

In [14]:
model.compile(loss=tf.keras.losses.LogCosh(), optimizer=tf.keras.optimizers.Adam())

model.summary()

NameError: name 'model' is not defined